In [210]:
import pandas as pd 
import numpy as np
import random

def bwt(p):
    """Burrows-Wheeler transform of a string"""
    if len(p)==0:
        return None
    s='$'+p
    rotations = list(enumerate([s[i:] + s[:i] for i in range(len(s))]))
    rotations.sort(key=lambda x: x[-1])
    df = pd.DataFrame([[rot[0],rot[1][0],rot[1][-1]] for rot in rotations])
    df.columns = ['index','L','R']

    
    letters = sorted(list(df['R'].unique()))
    rank = pd.DataFrame()
    for let in letters:
        rank[let] = (df['R']==let).cumsum() - (df['R']==let)
    
    rank.loc[len(s)]=df['R'].value_counts().sort_index()

    rng = pd.DataFrame([],index=letters)
    d = df['L'].value_counts().sort_index()
    rng['start']=d.cumsum() - d
    rng['end']=d.cumsum()
    
    return df ,rank,rng



def get_lmem(q,df,rank,rng):
    """Burrows-Wheeler search of a string"""
    n =len(q)
    st,end = rng.loc[q[-1]]
    j2 = n-1
    lmem_list=[]
    l_=5
    for j in range(n-2,-1,-1):
        q_ = q[j]
        #print(q_)
        st_ = rank.loc[st,q_]
        end_ = rank.loc[end,q_]
        
        if(st_==end_):
            #print('No match')
            j1 = j+1 
            l = j2-j1
            if(l>=l_-1):
                i1=df.iloc[st:end]['index'].values[0]-1
                #print(i1)
                i2 = i1 + l
                lmem_list.append((i1,i2+1,j1,j2+1,i1-j1))
            j2 = j
            st = rng.loc[q_,'start']
            end = rng.loc[q_,'end']

        elif j == 0:
            #print('Match 0')
            st = rng.loc[q_,'start'] + st_
            end = rng.loc[q_,'start'] + end_     
            j1 = j
            l = j2-j1
            if(l>=l_-1):
                i1=df.iloc[st:end]['index'].values[0]-1
                #print(i1)
                i2 = i1 + l
                lmem_list.append((i1,i2+1,j1,j2+1,i1-j1))

        else:
            st = rng.loc[q_,'start'] + st_
            end = rng.loc[q_,'start'] + end_
        
    return lmem_list
        

In [142]:
def get_colinear_sets(lmem_list):
    """Get colinear sets from lmem_list"""
    lmem_list.sort(key=lambda x: x[-1])
    thresh = 30
    colinear_sets = []
    l= [lmem_list[0]]
    for i in range(1,len(lmem_list)):
        if abs(lmem_list[i][-1]-lmem_list[i-1][-1])<=thresh:
            l.append(lmem_list[i])
        else:
            if(len(l)>1):
                colinear_sets.append(l)
            l = [lmem_list[i]]
    if len(l)>1:
        colinear_sets.append(l)
    return colinear_sets

In [217]:
def non_outlier(col_set):
    col_set.sort(key=lambda x: x[2])
    thresh = 10
    n=len(col_set)
    dp = np.ones(n)
    for i in range(1,n):
        for j in range(i-1,-1,-1):
            if abs(col_set[i][-1]-col_set[j][-1])<=thresh:
                dp[i] = max(dp[i],dp[j]+1)
    i = dp.argmax()
    non_outlier_set = [col_set[i]]
    for j in range(i-1,-1,-1):
        if (abs(col_set[i][-1]-col_set[j][-1])<=thresh) and (dp[j]==dp[i]-1):
            non_outlier_set.append(col_set[j])
            i=j
    non_outlier_set.reverse()
    return non_outlier_set


def remove_overlap(col_set):
    for i in range(len(col_set)-1):
        lmem1 = col_set[i]
        lmem2 = col_set[i+1]
        l1 = lmem1[1]-lmem1[0]
        l2 = lmem2[1]-lmem2[0]
        if lmem1[1]>lmem2[0]:
            print("Bazinga")
            if l1>l2:
                col_set[i+1] = [lmem1[1],lmem2[1],lmem2[2],lmem2[2]+lmem2[1]-lmem1[1],lmem2[2]-lmem1[1]]
            else:
                col_set[i] = [lmem1[0],lmem2[0],lmem1[2],lmem1[2]+lmem2[0]-lmem1[0],lmem1[4]]
    return col_set

def preprocess_colinear_sets(colinear_sets): 
    """Preprocess colinear sets"""
    for i in range(len(colinear_sets)):
        col_set = colinear_sets[i]
        col_set.sort(key=lambda x: x[2])
        col_set = non_outlier(col_set) #Remover outliers
        col_set = remove_overlap(col_set) #Remove overlap
        colinear_sets[i] = col_set
    return colinear_sets


#Write helper functions for gap filling and alignment here
        

In [218]:
p = ''.join([random.choice('ACGT') for i in range(10000)])
q = ''.join([random.choice('ACGT') for i in range(10000)])

df,rank,rng = bwt(p)
lmem_set = get_lmem(q,df,rank,rng)
colinear_sets = get_colinear_sets(lmem_set)
colinear_sets = preprocess_colinear_sets(colinear_sets)